<a href="https://colab.research.google.com/github/VengSW/train-model/blob/main/Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Fri Nov 12 13:18:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    25W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install tensorflow-gpu

     |████████████████████████████████| 489.6 MB 12 kB/s 


In [ ]:
import tensorflow as tf
tf.__version__

'2.7.0'

In [ ]:
#import the libraries
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [ ]:
#resize images
IMAGE_SIZE = [224,224]

train_path = '/content/drive/MyDrive/Datasets/train'
test_path = '/content/drive/MyDrive/Datasets/test'

In [ ]:
#imprt Vgg 16 library as shown below & add preprocessing layer to the front of VGG
#imagenet weights being used
inception = InceptionV3(input_shape=IMAGE_SIZE + [3],weights='imagenet',include_top=False)

87924736/87910968 [==============================] - 1s 0us/step


In [ ]:
#dont train existing weights
for layer in inception.layers:
  layer.trainable = False

In [ ]:
#useful for getting number of output classes
folders =glob('/content/drive/MyDrive/Datasets/train/*')

In [ ]:
folders

['/content/drive/MyDrive/Datasets/train/Good Sitting',
 '/content/drive/MyDrive/Datasets/train/Poor Sitting',
 '/content/drive/MyDrive/Datasets/train/Poor Sitting - knee crossed',
 '/content/drive/MyDrive/Datasets/train/Poor Sitting - Close distance']

In [ ]:
#our layers can add more
x = Flatten()(inception.output)

In [ ]:
prediction = Dense(len(folders),activation='softmax')(x)

#create a model object
model = Model(inputs=inception.input,outputs=prediction)

In [ ]:
#view structure of model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [ ]:
#tell the model what cost and optimization method to use
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
#use the image data generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range =0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
#make sure you provide the same target size as initiated for the image size
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Datasets/train',
                                                  target_size = (224,224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 105 images belonging to 4 classes.


In [ ]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Datasets/test',
                                                  target_size = (224,224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 105 images belonging to 4 classes.


In [ ]:
#fit the model
r = model.fit_generator(
    training_set,
    validation_data=test_set,
    epochs =10,
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set)
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


Epoch 1/10


UnknownError: ignored